In [14]:
# take in the url of a live video feed and use cv2 to capture the video feed, and save the keyframes to a rolling directory that maintains a certain number of keyframes (e.g. 1000)
# the keyframes are saved as jpgs
# the keyframes are saved in a rolling directory that maintains a certain number of keyframes (e.g. 1000)

import cv2
import os
import time
import datetime
import numpy as np
import pandas as pd
import requests
import json
import sys
import glob
import shutil

# read the feed urls from the file webcam_data_worldwide/links.txt

feed_list = pd.read_csv('links.csv', header=None)

# set the number of keyframes to maintain
num_keyframes = 1000

# set the directory to save the keyframes to
project_dir = '/Volumes/Backups of Grahams IMAC/PythonProjects/PySeas'
# for each feed in the feed list, create a directory to save the keyframes to
for feed in feed_list:
    # create the directory to save the keyframes to
    keyframe_dir = project_dir + '/keyframes/' + str(feed) # a directory for each feed
    # if the directory does not exist, create it
    if not os.path.exists(keyframe_dir):
        os.makedirs(keyframe_dir)




In [15]:
# make the first row of the feed_list df the column names
feed_list.columns = feed_list.iloc[0]
# drop the first row
feed_list = feed_list.drop(feed_list.index[0])
feed_list.head()

,Location of Webcam,webcam url
1,Hawaii,https://youtu.be/DY5RYp4sxYc
2,Santa Monica Beach,https://youtu.be/OWbI6WtlI-k
3,Waimea Bay Oahu Hawaii,https://youtu.be/wnNrd-VjLsQ
4,Galapagos Islands,https://youtu.be/1Z9Z5Z1Z1Zs
5,Deerfield Beach Florida USA,https://www.youtube.com/watch?v=-HAi_5IIAYg


Stackoverflow has an answer for how to ingest live youtube video.

```python
# pip install urllib
# pip install m3u8
# pip install streamlink
from datetime import datetime, timedelta, timezone
import urllib
import m3u8
import streamlink
import cv2 #openCV

def get_stream(url):

    """
    Get upload chunk url
    input: youtube URL
    output: m3u8 object segment
    """
    #Try this line tries number of times, if it doesn't work, 
    # then show the exception on the last attempt
    # Credit, theherk, https://stackoverflow.com/questions/2083987/how-to-retry-after-exception
    tries = 10
    for i in range(tries):
    try:
        streams = streamlink.streams(url)
    except:
        if i < tries - 1: # i is zero indexed
            print(f"Attempt {i+1} of {tries}")
            time.sleep(0.1) #Wait half a second, avoid overload
            continue
        else:
            raise
    break

    stream_url = streams["best"] #Alternate, use '360p'

    m3u8_obj = m3u8.load(stream_url.args['url'])
    return m3u8_obj.segments[0] #Parsed stream


def dl_stream(url, filename, chunks):
    """
    Download each chunk to file
    input: url, filename, and number of chunks (int)
    output: saves file at filename location
    returns none.
    """
    pre_time_stamp = datetime(1, 1, 1, 0, 0, tzinfo=timezone.utc)

    
    #Repeat for each chunk
    #Needs to be in chunks because 
    #  1) it's live
    #  2) it won't let you leave the stream open forever
    i=1
    while i <= chunks:
       
        #Open stream
        stream_segment = get_stream(url)
    
        #Get current time on video
        cur_time_stamp = stream_segment.program_date_time
        #Only get next time step, wait if it's not new yet
        if cur_time_stamp <= pre_time_stamp:
            #Don't increment counter until we have a new chunk
            print("NO   pre: ",pre_time_stamp, "curr:",cur_time_stamp)
            time.sleep(0.5) #Wait half a sec
            pass
        else:
            print("YES: pre: ",pre_time_stamp, "curr:",cur_time_stamp)
            print(f'#{i} at time {cur_time_stamp}')
            #Open file for writing stream
            file = open(filename, 'ab+') #ab+ means keep adding to file
            #Write stream to file
            with urllib.request.urlopen(stream_segment.uri) as response:
                html = response.read()
                file.write(html)
            
            #Update time stamp
            pre_time_stamp = cur_time_stamp
            time.sleep(stream_segment.duration) #Wait duration time - 1

            i += 1 #only increment if we got a new chunk

    return None

def openCVProcessing(saved_video_file):
    '''View saved video with openCV
    Add your other steps here'''
    capture = cv2.VideoCapture(saved_video_file)

    while capture.isOpened():
        grabbed, frame = capture.read()  #read in single frame
        if grabbed == False:
            break

        #openCV processing goes here
        #
        
        cv2.imshow('frame',frame)  #Show the frame
        
        #Shown in a new window, To exit, push q on the keyboard
        if cv2.waitKey(20) & 0xFF == ord('q'):
            break

    capture.release()
    cv2.destroyAllWindows()  #close the windows automatically
                  
tempFile = "temp.ts"  #files are format ts, open cv can view them
videoURL = "https://www.youtube.com/watch?v=_9OBhtLA9Ig"

dl_stream(videoURL, tempFile, 3)
openCVProcessing(tempFile)```

In [16]:
# Use the code above to scan the urls we have identified in the dataframe and save the keyframes to the rolling directory we have created.

# for each feed in the feed list, create a directory to save the keyframes to
for feed in feed_list:
    # create the directory to save the keyframes to
    keyframe_dir = project_dir + '/keyframes/' + str(feed) # a directory for each feed
    # if the directory does not exist, create it
    if not os.path.exists(keyframe_dir):
        os.makedirs(keyframe_dir)
    # get the url of the live feed
    url = feed_list[feed][0]
    # set the number of keyframes to maintain
    num_keyframes = 1000
    # set the directory to save the keyframes to
    project_dir = '/Volumes/Backups of Grahams IMAC/PythonProjects/PySeas'
    # set the directory to save the keyframes to
    keyframe_dir = project_dir + '/keyframes/' + str(feed) # a directory for each feed



SyntaxError: invalid syntax (1737403956.py, line 2)